# Modelo

La optimizacion la hago con el dataset undersampleado porque sino tarda una eternidad.

En el proximo script entreno ahi si con la totalidad de los datos.

In [1]:
import pandas as pd
import polars as pl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

import pickle

In [ ]:
# !gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_02_inflacion_adj_cpe.parquet /home/eanegrin/datasets/

Copying file:///home/eanegrin/buckets/b1/datasets/competencia_02_inflacion_adj_cpe.parquet...
- [1 files][829.6 MiB/829.6 MiB]                                                
Operation completed over 1 objects/829.6 MiB.                                    


In [3]:
# base_path = '/content/drive/MyDrive/DMEyF/2024/'
base_path = 'C:/Eugenio/Maestria/DMEyF/'
# base_path = '/home/eanegrin/buckets/b1/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_02_inflacion_adj_cpe_undersampled_exp.parquet'

ganancia_acierto = 273000
costo_estimulo = 7000

# agregue sus semillas
semillas = [122219, 109279, 400391, 401537, 999961]

# data = pd.read_parquet('/home/eanegrin/datasets/' + dataset_file)
data = pd.read_parquet(dataset_path + dataset_file)

In [4]:
data.shape

(410019, 353)

In [ ]:
# corrijo los tipos de estas 2 columnas que se guardaron como string en el parquet

# data[['tmobile_app', 'cmobile_app_trx']] = data[['tmobile_app', 'cmobile_app_trx']].astype('float')

In [5]:
meses_train = [201906, 201907, 201908, 201909, 201910, 201911, 201912,
               202001, 202002, 202003, 202004, 202005, 202006,
               202007, 202008, 202009, 202010, 202011, 202012,
               202101, 202102, 202103, 202104, 202105] # dejo afuera 202106 para test

data = data[data['foto_mes'].isin(meses_train)]
data.shape

(391550, 353)

In [6]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [7]:
data['clase_binaria'] = 0
data['clase_binaria'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

In [8]:
X_train = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)
y_train_binaria = data['clase_binaria'] # Junta a los 2 baja
w_train = data['clase_peso']

Para evaluar la calidad del modelo, crearemos nuestra propia función de evaluación que calcule la ganancia. La razón de incluir los pesos es precisamente para poder implementar esta función de evaluación de manera adecuada. Al combinar las clases *BAJA+1* y *BAJA+2* en una sola, necesitamos una forma de diferenciarlas, y es aquí donde entra en juego el *weight*. Este parámetro nos permitirá distinguir entre ambas clases al momento de evaluarlas dentro del algoritmo.


In [9]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

# Optimizacion

In [10]:
def objective(trial):

    num_leaves = trial.suggest_int('num_leaves', 8, 150),
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.05), # mas bajo, más iteraciones necesita
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 2000),
    feature_fraction = trial.suggest_float('feature_fraction', 0.1, 1.0),
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.1, 1.0),

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': semillas[0],
        'verbose': -1
    }
    
    train_data = lgb.Dataset(X_train,
                              label=y_train_binaria, # eligir la clase
                              weight=w_train)
    
    # print(f"Learning Rate: {learning_rate}, Type: {type(learning_rate)}")
    
    # Use callbacks for early stopping
    early_stopping_cb = lgb.early_stopping(stopping_rounds=50) # creo que en min_delta le tendria que pasar un parametro con la ganancia que considero irrelevante
        
    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=1000, # modificar, subit y subir... y descomentar la línea inferior (ahora le puso 100 para mostrarnos, pero hay que ponerle un numero alto, 10.000, 200.000)
        callbacks=[early_stopping_cb],
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0]
    )
    
    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5 # funcion objetivo, en el proximo paso le digo si quiero maximizarla o minimizarla.

In [11]:
db_path

'C:/Eugenio/Maestria/DMEyF/db/'

In [12]:
storage_name = "sqlite:///" + db_path + "optimization_experimento_cpe.db"
study_name = "experimento_cpe_v02" # UPDATE

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

[I 2024-11-20 11:17:34,347] A new study created in RDB with name: experimento_cpe_v02


In [13]:
study.optimize(objective, n_trials=50)

Training until validation scores don't improve for 50 rounds


[I 2024-11-20 11:20:20,098] Trial 0 finished with value: 3340953000.0 and parameters: {'num_leaves': 83, 'learning_rate': 0.039839431046083264, 'min_data_in_leaf': 1164, 'feature_fraction': 0.7804158768739993, 'bagging_fraction': 0.3890074093495989}. Best is trial 0 with value: 3340953000.0.


Did not meet early stopping. Best iteration is:
[998]	cv_agg's valid gan_eval: 6.68191e+08 + 8.17243e+06
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	cv_agg's valid gan_eval: 6.58011e+08 + 8.74925e+06


[I 2024-11-20 11:22:41,413] Trial 1 finished with value: 3290056000.0 and parameters: {'num_leaves': 44, 'learning_rate': 0.038768079602832475, 'min_data_in_leaf': 404, 'feature_fraction': 0.38101374750814543, 'bagging_fraction': 0.28717252164869533}. Best is trial 0 with value: 3340953000.0.


Training until validation scores don't improve for 50 rounds


...

Did not meet early stopping. Best iteration is:
[992]	cv_agg's valid gan_eval: 6.73138e+08 + 8.50745e+06
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	cv_agg's valid gan_eval: 6.64266e+08 + 8.18746e+06


[I 2024-11-20 13:47:59,297] Trial 48 finished with value: 3321332000.0 and parameters: {'num_leaves': 57, 'learning_rate': 0.04615040439097598, 'min_data_in_leaf': 1720, 'feature_fraction': 0.5722431364283942, 'bagging_fraction': 0.330090895574531}. Best is trial 41 with value: 3374434000.0.


Training until validation scores don't improve for 50 rounds


[I 2024-11-20 13:50:56,097] Trial 49 finished with value: 3370864000.0 and parameters: {'num_leaves': 128, 'learning_rate': 0.04813891552242928, 'min_data_in_leaf': 693, 'feature_fraction': 0.7570868158350086, 'bagging_fraction': 0.5475240520762206}. Best is trial 41 with value: 3374434000.0.


Did not meet early stopping. Best iteration is:
[1000]	cv_agg's valid gan_eval: 6.74173e+08 + 9.19209e+06


Analizamos los resultados as usual

In [14]:
optuna.visualization.plot_optimization_history(study)

In [15]:
plot_param_importances(study)

El **learning rate** es un parámetro que tiene que ir acompañado por más árboles.

In [16]:
plot_slice(study)

In [17]:
plot_contour(study)

In [18]:
plot_contour(study, params=['num_leaves','min_data_in_leaf'])

In [19]:
study.best_trial.params

{'num_leaves': 133,
 'learning_rate': 0.049944983741191065,
 'min_data_in_leaf': 1586,
 'feature_fraction': 0.5671480065546671,
 'bagging_fraction': 0.4968927114663101}

In [20]:
best_iter = study.best_trial.user_attrs["best_iter"]
best_iter

991